# Data Processing

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
import datetime as dt
import numpy as np

## 1. Load raw data
The dataset is the minute transaction data of the index NIFTY 100 for the past 3 years, obtained at https://www.kaggle.com/hk7797/stock-market-india

In [2]:
raw = pd.read_csv('../data/raw data/raw_NIFTY100.csv')
raw.head()

,timestamp,open,high,low,close,volume
0,2017-01-02 09:15:00+05:30,8409.60,8409.75,8390.75,8391.10,0.0
1,2017-01-02 09:16:00+05:30,8391.85,8395.35,8390.40,8391.15,0.0
2,2017-01-02 09:17:00+05:30,8391.15,8391.55,8373.60,8373.60,0.0
3,2017-01-02 09:18:00+05:30,8373.60,8376.75,8369.90,8376.75,0.0
4,2017-01-02 09:19:00+05:30,8376.95,8377.25,8372.60,8373.70,0.0


In [3]:
df = raw.copy()
df.timestamp = pd.to_datetime(df.timestamp)
df['date'] = df.timestamp.apply(lambda x: x.date)
df['minute'] = df.timestamp.apply(lambda x: x.time)

# Getting rid of dates where #data points are insufficient
a = pd.DataFrame(df.groupby('date').close.count())
drop_dates = list(a[a.close < 300].index)
df = df[~df.date.isin(drop_dates)]

df.head()

,timestamp,open,high,low,close,volume,date,minute
0,2017-01-02 09:15:00+05:30,8409.60,8409.75,8390.75,8391.10,0.0,2017-01-02,09:15:00
1,2017-01-02 09:16:00+05:30,8391.85,8395.35,8390.40,8391.15,0.0,2017-01-02,09:16:00
2,2017-01-02 09:17:00+05:30,8391.15,8391.55,8373.60,8373.60,0.0,2017-01-02,09:17:00
3,2017-01-02 09:18:00+05:30,8373.60,8376.75,8369.90,8376.75,0.0,2017-01-02,09:18:00
4,2017-01-02 09:19:00+05:30,8376.95,8377.25,8372.60,8373.70,0.0,2017-01-02,09:19:00


In [4]:
df.timestamp.min()

Timestamp('2017-01-02 09:15:00+0530', tz='pytz.FixedOffset(330)')

In [5]:
df.timestamp.max()

Timestamp('2021-01-01 15:29:00+0530', tz='pytz.FixedOffset(330)')

In [6]:
df.groupby('date').close.count().unique()

array([375, 336, 372, 374, 361, 373], dtype=int64)

## 2. First hour of the market
This function is for extracting closing prices during the first hour of market exchange

In [7]:
def first_hour(time):
    if time.hour == 9:
        if time.minute >=15:
            return True
    elif time.hour == 10:
        if time.minute < 15:
            return True
    else:
        return False

In [8]:
first = df.copy()
first['temp'] = first.minute.apply(first_hour)
first = first[first.temp==True]
first = first.drop(['timestamp', 'open', 'high', 'low', 'volume', 'temp'], axis=1)
first.head()

,close,date,minute
0,8391.10,2017-01-02,09:15:00
1,8391.15,2017-01-02,09:16:00
2,8373.60,2017-01-02,09:17:00
3,8376.75,2017-01-02,09:18:00
4,8373.70,2017-01-02,09:19:00


In [9]:
first.groupby('date').close.count().unique()

array([60], dtype=int64)

- store the dataframe as csv file

In [10]:
first.to_csv('../data/processed data/first_combined.csv',index=False)

In [11]:
df.shape, first.shape

((370500, 8), (59280, 3))

## 3. Labeling; Whether the price goes up or down that day
- If the price goes up, label as 1
- If the price goes down or does not change, label as 0

In [12]:
label = pd.DataFrame()
for date in df.date.unique():
    day = df[df.date==date]
    day = day.sort_values('minute')
    first_open = day.iloc[0].open
    last_close = day.iloc[-1].close
    temp = pd.DataFrame([[date, first_open, last_close]])
    label = label.append(temp)


label.columns = ['date','open','close']
label['diff'] = label.close - label.open

def label_convert(diff):
    if diff > 0:
        return '1'
    else:
        return '0'
    
label['label'] = label['diff'].apply(label_convert)
label = label.drop(['open','close','diff'],axis=1)
label.head()

,date,label
0,2017-01-02,0
0,2017-01-03,1
0,2017-01-04,0
0,2017-01-05,1
0,2017-01-06,0


## 4. Store data as cvs file

In [13]:
path = '../data/processed data'
label.to_csv(path+'/label_dir_2.csv', index=False)